In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

import warnings
warnings.filterwarnings(action='ignore')

## Load Data

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")
train_df_origin = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test_df_origin = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")

In [ ]:
train_df_origin = train_df_origin.drop(['id'], axis=1)
train_df_origin.head(2)

In [ ]:
## have no target column

test_df_origin.head(2)

## Variables type check 
- numeric  vs  categorical

In [ ]:
temp_df = test_df_origin.copy()
temp_df = temp_df.drop(["id"], axis=1)
total_features = temp_df.columns.to_list()
numeric_cols = []
categorical_cols = []

for col in total_features:
    if len(train_df_origin[col].unique()) <3:
        categorical_cols.append(col)
    else:
        numeric_cols.append(col)

In [ ]:
len(numeric_cols), len(categorical_cols) ## variables are all numeric

In [ ]:
len(numeric_cols) + len(categorical_cols) == 100

## Split train/validation set

In [ ]:
## check target feature's distribution

train_df_origin['target'].value_counts()

In [ ]:
train_df, validation_df = train_test_split(train_df_origin, test_size=0.2, random_state=42)

In [ ]:
trainX = train_df[numeric_cols].values

validationX = validation_df[numeric_cols].values

trainY = train_df['target']
validationY = validation_df['target']

In [ ]:
trainX.shape, trainY.shape, validationX.shape, validationY.shape

## ANN modeling

In [ ]:
model = Sequential([
    Dense(256, input_shape=(100,), activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["AUC"])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
history = model.fit(trainX, trainY, epochs=100, batch_size=512, validation_data=(validationX, validationY))

## Visualize history

In [ ]:
def plot_history(history):
    auc = history.history['auc']
    val_auc = history.history['val_auc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(auc))

    plt.plot(epochs, auc, 'bo' ,label = 'training auc')
    plt.plot(epochs, val_auc, 'r' , label= 'validation auc')
    plt.title('Training and Validation auc')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo' ,label = 'training loss')
    plt.plot(epochs, val_loss, 'r' , label= 'validation loss')
    plt.title('Training and Validation loss')
    plt.legend()

In [ ]:
plot_history(history)

## Predict & make submission table

In [ ]:
## see sample submission format

sample_df = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv')

In [ ]:
sample_df.head()

In [ ]:
test_df_origin.head()

In [ ]:
testX_df = test_df_origin.drop(['id'], axis=1)
testX = testX_df.values

In [ ]:
y_pred = model.predict(testX).reshape(1, -1)[0]

In [ ]:
len(test_df_origin) == len(y_pred)

In [ ]:
submission_df = pd.DataFrame().from_dict({'id': test_df_origin['id'], 'target': y_pred})

In [ ]:
submission_df.head()

In [ ]:
# submission_df.to_csv("/kaggle/input/submission_yj.csv")